In [1]:
import pandas as pd
import numpy as np
import requests

In [3]:
import pandas as pd
import requests

# OpenFDA API data for state GDP
def get_state_gdp_data(state_code):
    url = 'https://api.fda.gov/openfda/drug/ndc_product.json?search=state:"' + state_code + '"&limit=1000'
    response = requests.get(url)
    data = response.json()['results']

    total_state_gdp = 0
    for product in data:
        if 'gdp_estimate' in product['results'][0]['openfda']:
            total_state_gdp += product['results'][0]['openfda']['gdp_estimate']

    return {
        'state': state_code,
        'gdp': total_state_gdp
    }

# USDA API data for major exports
def get_state_major_export_data(state_code):
    url = 'https://apps.fas.usda.gov/export/ag_trade_insights/states/state_highlights.ashx?state_code=' + state_code
    response = requests.get(url)
    data = response.json()

    major_export = data['top_ag_exports'][0]['name']
    export_value = data['top_ag_exports'][0]['value']

    return {
        'state': state_code,
        'major_export': major_export,
        'export_value': export_value
    }

# EPA API data for carbon emissions
def get_state_carbon_emissions_data(state_code):
    url = 'https://api.openei.org/v1/datasets/EPA/ghg_inventory_2020/data?state=' + state_code
    response = requests.get(url)
    data = response.json()['result']

    sector_emissions = 0
    for sector in data:
        if sector['sector'] == 'Electric Power Generation':
            sector_emissions = sector['emissions']

    return {
        'state': state_code,
        'emissions': sector_emissions
    }

# Main function to gather data and create DataFrame
def gather_and_compile_data():
    df = pd.DataFrame(columns=['State', 'GDP_Percentage', 'Major_Export', 'Emissions_Percentage'])

    # Total US GDP for percentage calculation
    total_us_gdp_response = requests.get('https://www.bea.gov/news/2023/national-income-and-product-accounts-gdp-third-quarter-2023')
    total_us_gdp = float(total_us_gdp_response.text.split('Total')[1].split('$')[1].replace(',', ''))

    # Total US carbon emissions for percentage calculation
    total_us_emissions_response = requests.get('https://www.eia.gov/environment/emissions/co2-emissions/overview/')
    total_us_emissions = float(total_us_emissions_response.text.split('Total')[1].split('thousand')[0].replace(',', ''))

    try:
        total_us_gdp = float(total_us_gdp_response.text.split('Total')[1].split('$')[1].replace(',', ''))
    except:  # Handle potential errors
        print("Error extracting GDP data")
        continue  # Skip to the next iteration

    # Total US carbon emissions for percentage calculation
    total_us_emissions_response = requests.get('https://www.eia.gov/environment/emissions/co2-emissions/overview/')
    print(total_us_emissions_response.text)  # Print the response's text for debugging

    try:
        total_us_emissions = float(total_us_emissions_response.text.split('Total')[1].split('thousand')[0].replace(',', ''))
    except:  # Handle potential errors
        print("Error extracting carbon emissions data")
        continue  # Skip to the next iteration


    for state_code in ['CA', 'NY', 'TX', 'FL', 'IL']:
        state_gdp_data = get_state_gdp_data(state_code)
        state_major_export_data = get_state_major_export_data(state_code)
        state_emissions_data = get_state_carbon_emissions_data(state_code)

        row = {
            'State': state_gdp_data['state'],
            'GDP_Percentage': (state_gdp_data['gdp'] / total_us_gdp) * 100,
            'Major_Export': state_major_export_data['major_export'],
            'Emissions_Percentage': (state_emissions_data['emissions'] / total_us_emissions) * 100
        }

        df = df.append(row, ignore_index=True)

    return df

# Gather data and create DataFrame
df = gather_and_compile_data()

IndexError: list index out of range